In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load credentials from .env file
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Create database connection URL (for MySQL; change driver if needed)
db_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(db_url)

In [2]:
# Step 1: Load base tables
financialrequestinstallments = pd.read_sql(
    "SELECT FinancialRequestId, AcutalPaidAmount FROM financialrequestinstallments", engine
)
financialrequests = pd.read_sql(
    "SELECT Id, CreatorId FROM financialrequests", engine
)
abpusers = pd.read_sql(
    "SELECT Id, Name, ManagerId FROM abpusers", engine
)

In [3]:
# Step 2: Join installments → requests
df = financialrequestinstallments.merge(
    financialrequests,
    left_on="FinancialRequestId",
    right_on="Id",
    how="inner"
).drop(columns=["Id"])  # drop duplicate Id from requests
df

,FinancialRequestId,AcutalPaidAmount,CreatorId
0,08dd3bb0-a253-415b-8fb0-f181c814eafc,0.0,None
1,08dd3bb0-a253-415b-8fb0-f181c814eafc,0.0,None
2,08dd3bb0-a253-415b-8fb0-f181c814eafc,0.0,None
3,08dd3bb0-a253-415b-8fb0-f181c814eafc,0.0,None
4,08dd3bb0-a253-415b-8fb0-f181c814eafc,0.0,None
...,...,...,...
851,08ddf138-d6ce-450f-8a45-83c4f1ff6c13,0.0,None
852,08ddf138-d6ce-450f-8a45-83c4f1ff6c13,0.0,None
853,08ddf138-d6ce-450f-8a45-83c4f1ff6c13,0.0,None
854,08ddf138-d6ce-450f-8a45-83c4f1ff6c13,0.0,None


In [4]:
# Step 3: Join with users (officers)
df = df.merge(
    abpusers,
    left_on="CreatorId",
    right_on="Id",
    how="inner",
    suffixes=("", "_user")
)
df

,FinancialRequestId,AcutalPaidAmount,CreatorId,Id,Name,ManagerId


In [5]:
# Step 4: Ensure numeric type for AcutalPaidAmount
df["AcutalPaidAmount"] = pd.to_numeric(df["AcutalPaidAmount"], errors="coerce").fillna(0)
df

,FinancialRequestId,AcutalPaidAmount,CreatorId,Id,Name,ManagerId


In [6]:
# Step 5: Group by officer
agg_df = df.groupby(
    ["CreatorId", "Name", "ManagerId"], as_index=False
).agg(
    TotalCollectedAmount=("AcutalPaidAmount", "sum")
)
agg_df

,CreatorId,Name,ManagerId,TotalCollectedAmount


In [7]:
# Step 6: Rename / format columns
final_df = agg_df.rename(columns={
    "Name": "OfficerName",
    "CreatorId": "OfficerId",
    "ManagerId": "ManagerId"
}).sort_values("TotalCollectedAmount", ascending=False).reset_index(drop=True)
final_df

,OfficerId,OfficerName,ManagerId,TotalCollectedAmount
